## 코드 추상화: 클래스와 객체 2부

안내: [python-textbook.readthedocs.io의 Classes](https://python-textbok.readthedocs.io/en/1.0/Classes.html#class-attributes) 내용을 요약 및 수정한 내용입니다.

## 클래스 장식자

## 매직 메서드

## 매직 메서드 재정의